In [1]:
# ruff: noqa: F401

In [2]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [3]:
import sys

sys.path.insert(0, "..")

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from datetime import datetime, timedelta
from pathlib import Path

import hvplot.polars
import numpy as np
import polars as pl
from sqlalchemy import text

In [6]:
from tsdb_benchmarks.dbs.utils import drop_table

In [7]:
from tsdb_benchmarks.dbs.clickhouse import Clickhouse
from tsdb_benchmarks.dbs.duckdb import DuckDB
from tsdb_benchmarks.dbs.monetdb import MonetDB
from tsdb_benchmarks.dbs.monetdb.settings import SETTINGS as MONETDB_SETTINGS
from tsdb_benchmarks.dbs.timescaledb import TimescaleDB
from tsdb_benchmarks.settings import REPO_ROOT, setup_stdout_logging

setup_stdout_logging()

monetdb = MonetDB()
duckdb = DuckDB()
clickhouse = Clickhouse()
timescaledb = TimescaleDB()

# RTA bench has heavy queries that do not return much data, avoid using binary
# fetch since this emits a "... limit 1" query to determine the result schema
MONETDB_SETTINGS.default_fetch_method = "pymonetdb"

In [8]:
queries_dir = REPO_ROOT / "tsdb_benchmarks/suites/time_series/queries"


def load_query(name: str, db: str) -> str:
    with (queries_dir / f"{db}/{name}.sql").open() as f:
        return f.read()

In [9]:
from tsdb_benchmarks.suites.time_series.config import get_time_series_input_files

get_time_series_input_files()

{'data_small_wide': PosixPath('/Users/williamlauren/repos/tsdb-benchmarks/data/input/time_series/data_wide_0.2M_0.1k.parquet'),
 'data_large_wide': PosixPath('/Users/williamlauren/repos/tsdb-benchmarks/data/input/time_series/data_wide_2.0M_0.1k.parquet'),
 'data_small_eav': PosixPath('/Users/williamlauren/repos/tsdb-benchmarks/data/input/time_series/data_eav_0.2M_0.1k.parquet'),
 'data_large_eav': PosixPath('/Users/williamlauren/repos/tsdb-benchmarks/data/input/time_series/data_eav_2.0M_0.1k.parquet')}

In [10]:
df_small = pl.read_parquet("../data/input/time_series/data_wide_0.2M_0.1k.parquet")

In [11]:
df_large = pl.read_parquet("../data/input/time_series/data_wide_2.0M_0.1k.parquet")

In [12]:
df_large

time,process_4,binary_3,process_18,deviation_3,process_77,binary_5,process_80,process_5,process_16,process_2,ratio_1,process_75,process_11,process_20,process_9,process_60,deviation_1,process_58,process_36,ratio_2,process_82,process_63,process_28,process_23,process_31,process_40,process_10,process_42,binary_4,process_41,process_59,process_45,process_1,process_34,process_35,process_47,…,process_73,process_29,process_44,process_53,process_27,process_33,process_21,process_67,process_14,process_25,process_69,process_55,process_70,process_6,process_85,deviation_4,process_65,process_19,process_22,process_46,process_57,process_50,deviation_5,process_79,ratio_5,process_52,process_81,ratio_4,process_13,process_17,process_61,process_8,process_78,process_32,binary_2,process_30,process_71
datetime[ms],f32,bool,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,bool,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,bool,f32,f32
2021-03-14 02:41:00,485.821716,true,1108.630493,47.008812,131.541687,true,548.409302,658.197144,314.258911,842.423462,0.432263,970.02533,680.001526,937.692627,304.989441,268.193756,2.547028,717.441711,82.419861,0.386208,319.937653,1177.31958,511.813354,46.82671,378.628143,568.57196,621.558716,287.461761,true,324.238342,1002.618347,92.309639,935.960083,353.983185,631.037903,1199.821411,…,53.613346,384.458649,172.204636,1219.653198,757.058655,722.962097,47.651703,336.545135,631.689331,859.0672,276.390015,362.175964,603.493042,219.548126,228.796921,-27.712986,161.994293,641.684814,120.486954,857.649109,439.147522,137.868835,20.741936,65.606194,0.80898,1067.872314,856.603271,0.626373,216.607437,115.473526,895.805359,52.682476,55.478359,604.682983,true,413.30658,374.90799
2021-03-14 02:42:00,485.822601,true,1108.517578,-34.569168,131.363495,true,548.425171,658.196045,314.10498,842.657593,0.339784,971.118103,679.955627,937.423706,305.102966,268.276947,14.069798,717.573547,82.558601,0.217332,319.788513,1176.930786,512.149902,47.15242,378.741669,569.078552,621.689819,287.539368,false,324.29538,1002.635071,92.434975,935.391174,353.669739,630.695312,1200.146851,…,53.431297,384.512146,171.921906,1219.351807,757.070862,722.926086,47.649639,336.409546,631.615723,858.490601,276.977356,362.256927,603.441833,219.450958,228.641769,20.682272,162.05397,641.497192,120.39119,857.812805,438.69223,137.964325,-24.241034,65.515366,0.864773,1067.831055,856.840942,0.492569,216.316544,115.279549,895.531555,52.473701,55.354897,604.448364,true,413.428009,374.858337
2021-03-14 02:43:00,485.916626,true,1108.944336,-38.76989,131.48494,true,548.638062,657.918213,314.416901,842.698608,0.60675,971.241333,679.836304,937.723572,305.037048,268.313202,11.697422,718.093323,82.366768,0.343206,319.917633,1177.300293,511.783051,47.128326,379.076538,568.971802,621.57666,287.479523,false,324.448944,1002.581665,92.515366,935.906921,353.442596,630.907166,1200.220703,…,53.703094,384.858032,172.097549,1219.702026,757.216125,722.616516,47.696045,336.404053,631.560608,858.858826,276.808594,362.283081,603.507019,219.595291,228.838699,23.453012,162.228119,641.705444,120.533836,857.660828,438.821106,138.23999,4.194476,65.178398,0.392867,1067.305664,856.40155,0.546515,216.773361,114.921165,895.774719,52.832581,55.4002,604.172668,true,413.444183,375.21283
2021-03-14 02:44:00,485.618896,true,1109.101807,-35.69651,131.327667,true,548.372864,657.949036,314.267273,842.524963,0.277884,971.627319,680.019836,937.799316,304.937836,268.099426,-8.082939,717.5849,82.508179,0.591075,319.723877,1177.268921,511.701691,46.971649,379.070007,568.8526,621.675659,287.529694,false,324.61322,1002.936768,92.519821,936.021057,353.82373,630.760071,1200.13501,…,53.570976,384.619415,171.865982,1219.033203,756.975159,722.869812,47.637463,336.543335,631.512878,858.918213,276.912598,362.21109,603.556885,219.526901,228.868668,-18.21963